Объединенный датасет [FIX MEне доступен по ссылке](*).

Положить в папку data содержимое так, чтобы были следующие пути:  
* \$(ROOT_DIR)/data/full-rtsd/...
* \$(ROOT_DIR)/data/full-gt.csv

> *gt_Set_NaN.csv - содержит тот же датасет, но значения колонки Set обнулено*

gt - датафрейм содержащий:  
* имена файлов - поле filename
* класс знака - поле sign_class
* координаты знаков
* в какой набор включен знак - поле Set $\in$ $\{train, valid, test\}$  

In [ ]:
import matplotlib.pyplot as plt

import numpy as np
import random
import seaborn as sns
import pandas as pd
import os
import pathlib
import shutil
import cv2
import sys

%cd adas_system/notebooks

try:
    USE_TPU = bool(os.environ['COLAB_TPU_ADDR'])
except:
    USE_TPU = False

if USE_TPU:
    # !pip uninstall pytorch
    # !pip install cloud-tpu-client==0.10 torch==1.10.0
    # !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
    !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
    import torch_xla
    import torch_xla.core.xla_model as xm
    USE_TPU = True

else:
    USE_TPU = False

import torch
from torch import nn

try:
    import google.colab
    IN_COLAB = True
    from google.colab import drive
    drive.mount('/content/drive')
    if not os.path.isfile('1_ClassifierResearch.ipynb'):
        print('already exist')
        !git clone --branch 9_SignDetector https://github.com/lsd-maddrive/adas_system.git
        %cd adas_system/notebooks
        !mkdir ../data/rtsd-frames
        !unzip -j -q /content/drive/MyDrive/USER_FULL_FRAMES.zip -d ./../data/rtsd-frames
        !pwd
        !ls

except:
    IN_COLAB = False

###
import nt_helper
from nt_helper.helper_utils import *
###

TEXT_COLOR = 'black'

# Зафиксируем состояние случайных чисел
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)
random.seed(RANDOM_STATE)
%matplotlib inline
plt.rcParams["figure.figsize"] = (17,10)

if USE_TPU:
    device = xm.xla_device()
else:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
torch.__version__

## Init dirs

In [ ]:
if not IN_COLAB:
    PROJECT_ROOT = pathlib.Path(os.path.join(os.curdir, os.pardir))
else:
    PROJECT_ROOT = pathlib.Path('..')
    
DATA_DIR = PROJECT_ROOT / 'data'
NOTEBOOKS_DIR = PROJECT_ROOT / 'notebooks'

if (NOTEBOOKS_DIR / 'full-gt.csv').is_file():
    full_gt = pd.read_csv(NOTEBOOKS_DIR / 'full-gt.csv')
else:
    full_gt = pd.read_csv(DATA_DIR / 'full-gt.csv')

FORMATED_GT_PATH = "formated_full_gt.csv"
FULL_GT_SRC_LEN = len(full_gt.index)

## Init dataset DataFrame

In [ ]:
if os.path.isfile(FORMATED_GT_PATH):
    print("FORMATED GT EXIST. LOAD IT")
    
    import ast
    
    formated_full_gt_df = pd.read_csv(FORMATED_GT_PATH, dtype=object)
    formated_full_gt_df['coords'].replace({'\n ':',', ' \s+': ' ', '\[ ': '['}, regex=True, inplace=True)
    
    formated_full_gt_df['coords'] = formated_full_gt_df['coords'].apply(
        lambda x: ast.literal_eval(x)
    )
    
    formated_full_gt_df['size'] = formated_full_gt_df['size'].apply(
        lambda x: ast.literal_eval(x)
    )

    formated_full_gt_df['filepath'] = formated_full_gt_df['filepath'].apply(
        lambda x: x.replace('\\', '/')
    )
else:
    print("FORMATED GT DOESNT EXIST. CREATE IT")
    
    # get all original filenames
    full_gt_unique_filenames = set(full_gt['filename'])
    full_gt_unique_filenames_size = len(full_gt_unique_filenames)
    
    formated_full_gt_list = []

    import imagesize
    i = 0
    for src_filename_iterator in list(full_gt_unique_filenames):

        mask = np.in1d(full_gt['filename'], [src_filename_iterator])
        coord_data_arr = full_gt[mask][['x_from', 'y_from', 'width', 'height']].to_numpy()
        
        filepath = DATA_DIR / "rtsd-frames" / src_filename_iterator
        origW, origH = imagesize.get(filepath)
                
        rel_coord = []
        for coord in coord_data_arr:
            # make from x, y, dx, dx -> x1, y1, x2, y2
            CV2RectangleCoords = ConvertAbsTLWH2CV2Rectangle(coord)
   
            # make from x1, y1, x2, y2 -> x, y, w, h
            CV2CircleCoords = ConvertCV2Rectangle2CenterXYWH(CV2RectangleCoords)
            
            # make x, y, w, h -> relative x, y, w, h
            rel_instance = MakeRel(CV2CircleCoords, origW, origH)
            rel_coord.append(rel_instance)
            
        if i % 100 == 0:
            printProgressEnum(i, full_gt_unique_filenames_size)
        i += 1

        formated_full_gt_list.append([str(filepath), rel_coord, [origW, origH]])

    formated_full_gt_df = pd.DataFrame(formated_full_gt_list, columns=['filepath', 'coords', 'size'])
    formated_full_gt_df.to_csv("formated_full_gt.csv", index=False)

if 'set' in formated_full_gt_df.columns:
    print('SET ALREADY EXIST')
else:
    print('SET DOESNT EXIST. LETS CREATE IT')
    formated_full_gt_df_index_count = len(formated_full_gt_df.index)
    TRAIN_SIZE = round(0.7 * formated_full_gt_df_index_count)
    VALID_SIZE = round(0.2 * formated_full_gt_df_index_count)
    TEST_SIZE = round(formated_full_gt_df_index_count - TRAIN_SIZE - VALID_SIZE)
        
    assert TRAIN_SIZE + VALID_SIZE + TEST_SIZE == formated_full_gt_df_index_count, 'wrong split'
    set_series = pd.Series('test', index=range(TEST_SIZE)).append(
        pd.Series('train', index=range(TRAIN_SIZE)).append(
            pd.Series('valid', index=range(VALID_SIZE))
        )
    ).sample(frac=1).reset_index(drop=True)
    formated_full_gt_df['set'] = set_series
    formated_full_gt_df.to_csv("formated_full_gt.csv", index=False)
    
display(formated_full_gt_df.head())

# simple test

In [ ]:
instance = formated_full_gt_df.iloc[15466]

path_ = str(instance['filepath'])
img = cv2.imread(path_)
img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

h, w = img.shape[0], img.shape[1]
print('Shape:', w, h)

for i in instance['coords']:
    
    xywh = UnmakeRel(i, w, h)
    x1y1x2y2 = ConvertCenterXYWH2CV2Rectangle(xywh)
    print('+', MakeRel(x1y1x2y2, w, h))
    print('xywh', xywh)
    print('x1y1x2y2', x1y1x2y2)
    
    
    img = cv2.rectangle(img, (x1y1x2y2[0], x1y1x2y2[1]), 
                        (x1y1x2y2[2], x1y1x2y2[3]), 
                        (255, 0, 0), 
                        3)
    
    img = cv2.circle(img, 
                     (xywh[0], xywh[1]), 
                     xywh[2] // 2, 
                     (255, 255, 0), 
                     3)

plt.figure(figsize = (20, 20))  
plt.imshow(img)
plt.show()

# Now we have pd.DataFrame that contains filenames, list of relative coordinates, corresponding photo resoulutions and marks for set. 
## createDataLoaderAndDataSet function in utils.ipynb

In [ ]:
import yaml

hyps_file = "hyp.scratch.yaml"
with open(hyps_file, errors='ignore') as f:
    hyp = yaml.safe_load(f)


IMG_SIZE = 640
batch_size = 160
train_loader, train_dataset = createDataLoaderAndDataSet(formated_full_gt_df, 
                                                         'train',
                                                         hyp_arg=hyp,
                                                         imgsz=IMG_SIZE, 
                                                         batch_size=batch_size, 
                                                         augment=False)

test_loader, test_dataset = createDataLoaderAndDataSet(formated_full_gt_df, 
                                                         'test',
                                                         hyp_arg=hyp,
                                                         imgsz=IMG_SIZE, 
                                                         batch_size=batch_size, 
                                                         augment=True)

img, labels_out, filepath, shapes = train_dataset[6]
img_, labels_out_, filepath_, shapes_ = test_dataset[random.randrange(0, len(test_dataset))]

imgNT = img.numpy().transpose(1, 2, 0).astype(np.uint8).copy() #, cv2.COLOR_BGR2RGB)
print(labels_out)
print(filepath)
for coord in labels_out[:, 2:]:
    # print(coord)
    h, w = shapes[0]
    xywh = UnmakeRel(coord, IMG_SIZE, IMG_SIZE)
    x1y1x2y2 = ConvertCenterXYWH2CV2Rectangle(xywh)
    print(x1y1x2y2)
    imgNT = cv2.rectangle(imgNT, (x1y1x2y2[0], x1y1x2y2[1]), 
                        (x1y1x2y2[2], x1y1x2y2[3]), 
                        (255, 0, 0), 
                        3)


plt.figure(figsize = (20, 20))  
plt.imshow(imgNT)
plt.show()

ROAD SIGN ANCHORS: 10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326 src: https://grechka.family/dmitry/blog/2019/09/yolo-v3-anchors-for-traffic-sign-detection/

# Train?

In [ ]:
from torch.optim import SGD, Adam, AdamW, lr_scheduler
from torch.cuda import amp
from utils.general import one_cycle, LOGGER
from utils.loss import ComputeLoss
from utils.torch_utils import ModelEMA, de_parallel
from tqdm import tqdm
from datetime import datetime
from models.yolo import Model

import yaml

hyps_file = "hyp.scratch.yaml"
with open(hyps_file, errors='ignore') as f:
    hyp = yaml.safe_load(f)

def train(epochs, train_loader, test_loader, device, opt=None, imgsz=640, model=None, restore=None, model_cfg_path=None):
    
    if not model and not restore:
        print("ARG ERR NOT MODEL NOT RESTORE")
        return
    if restore:
        if model_cfg_path:         
            model_cfg_file = model_cfg_path
            model = Model(cfg=model_cfg_file, ch=3, nc=1)
            if os.path.isfile(restore):
                print('[+] restore passed, loading it')
                model.load_state_dict(torch.load(restore, map_location=device))
            else:
                print('[-] restore passed, but doesnt exist. just train')
        else:
            print("ARG ERR CFG PATH")
            return
    
    model.to(device)

    ###
    start_epoch = 0
    nc = 1
    # print(device.type)
    cuda = device.type == 'cuda'
    # print(cuda)
    nb = len(train_loader)
    nw = max(round(hyp['warmup_epochs'] * nb), 1000)
    nbs = 64  # nominal batch size
    batch_size = train_loader.batch_size
    last_opt_step = -1
    ###
        
    g0, g1, g2 = [], [], []  # optimizer parameter groups
    for v in model.modules():
        if hasattr(v, 'bias') and isinstance(v.bias, nn.Parameter):  # bias
            g2.append(v.bias)
        if isinstance(v, nn.BatchNorm2d):  # weight (no decay)
            g0.append(v.weight)
        elif hasattr(v, 'weight') and isinstance(v.weight, nn.Parameter):  # weight (with decay)
            g1.append(v.weight)
    
    if opt==None:
        optimizer = SGD(g0, lr=hyp['lr0'], momentum=hyp['momentum'], nesterov=True)
    else:
        if opt.optimizer == 'Adam':
            optimizer = Adam(g0, lr=hyp['lr0'], betas=(hyp['momentum'], 0.999))  # adjust beta1 to momentum
        elif opt.optimizer == 'AdamW':
            optimizer = AdamW(g0, lr=hyp['lr0'], betas=(hyp['momentum'], 0.999))  # adjust beta1 to momentum
        else:
            optimizer = SGD(g0, lr=hyp['lr0'], momentum=hyp['momentum'], nesterov=True)
    
    optimizer.add_param_group({'params': g1, 'weight_decay': hyp['weight_decay']})  # add g1 with weight_decay
    optimizer.add_param_group({'params': g2})  # add g2 (biases)
    del g0, g1, g2
    

    lf = one_cycle(1, hyp['lrf'], epochs)  # cosine 1->hyp['lrf']
    scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lf)
    
    ema = None # ModelEMA(model)
    
    nl = de_parallel(model).model[-1].nl  # number of detection layers (to scale hyps)
    hyp['box'] *= 3 / nl  # scale to layers
    hyp['cls'] *= nc / 80 * 3 / nl  # scale to classes and layers
    hyp['obj'] *= (imgsz / 640) ** 2 * 3 / nl  # scale to image size and layers
    hyp['label_smoothing'] = opt.label_smoothing if opt else 0.
    
    model.nc = nc  # attach number of classes to model
    model.hyp = hyp  # attach hyperparameters to model
    model.names = ['sign']
    
    scaler = amp.GradScaler(enabled=cuda)
    compute_loss = ComputeLoss(model)
    
    for epoch in range(start_epoch, epochs):
        model.train()
        mloss = torch.zeros(3, device=device)
        
        pbar = enumerate(train_loader)
        LOGGER.info(('\n' + '%10s' * 7) % ('Epoch', 'gpu_mem', 'box', 'obj', 'cls', 'labels', 'img_size'))
        pbar = tqdm(pbar, total=nb, bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}')  # progress bar
        
        optimizer.zero_grad()

        for i, (imgs, targets, paths, _) in pbar:
            ni = i + nb * epoch  # number integrated batches (since train start)
            imgs = imgs.to(device, non_blocking=True).float() / 255  # uint8 to float32, 0-255 to 0.0-1.0
            
            # Warmup
            if ni <= nw:
                xi = [0, nw]  # x interp
                # compute_loss.gr = np.interp(ni, xi, [0.0, 1.0])  # iou loss ratio (obj_loss = 1.0 or iou)
                accumulate = max(1, np.interp(ni, xi, [1, nbs / batch_size]).round())
                for j, x in enumerate(optimizer.param_groups):
                    # bias lr falls from 0.1 to lr0, all other lrs rise from 0.0 to lr0
                    x['lr'] = np.interp(ni, xi, [hyp['warmup_bias_lr'] if j == 2 else 0.0, x['initial_lr'] * lf(epoch)])
                    if 'momentum' in x:
                        x['momentum'] = np.interp(ni, xi, [hyp['warmup_momentum'], hyp['momentum']])

            # Forward
            with amp.autocast(enabled=cuda):
                pred = model(imgs)  # forward
                # print(pred[0].shape)
                # return pred, targets, paths
                loss, loss_items = compute_loss(pred, targets.to(device))  # loss scaled by batch_size
                
                
            # Backward
            scaler.scale(loss).backward()

            # Optimize
            if ni - last_opt_step >= accumulate:
                scaler.step(optimizer)  # optimizer.step
                scaler.update()
                optimizer.zero_grad()
                if ema:
                    ema.update(model)
                last_opt_step = ni
            
            if True:
                mloss = (mloss * i + loss_items) / (i + 1)  # update mean losses
                mem = f'{torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0:.3g}G'  # (GB)
                pbar.set_description(('%10s' * 2 + '%10.4g' * 5) % (
                    f'{epoch}/{epochs - 1}', mem, *mloss, targets.shape[0], imgs.shape[-1]))
        
        torch.cuda.empty_cache()
        
        now = datetime.now()
        model_save_name = 'YoloV5_{}_lbox{:.4f}_lobj{:.4f}.pt'.format(
            now.strftime("%d.%m_%H.%M"),
            mloss[0], mloss[1]
        )
        
        torch.save(model.state_dict(), model_save_name)
        torch.save(model.state_dict(), 'YoloV5Last.pt')
        if IN_COLAB:
            shutil.copy2(model_save_name, '/content/drive/MyDrive/')

        torch.save(model.state_dict(), 'YoloV5Last.pt')
        if IN_COLAB:
            shutil.copy2('YoloV5Last.pt', '/content/drive/MyDrive/')
    
    return model

torch.cuda.empty_cache()


IMG_SIZE = 640
batch_size = 80
num_workers = 4
train_loader, train_dataset = createDataLoaderAndDataSet(formated_full_gt_df, 
                                                         'train',
                                                         hyp_arg=hyp,
                                                         imgsz=IMG_SIZE, 
                                                         batch_size=batch_size, 
                                                         augment=True,
                                                         nw=num_workers)

test_loader, test_dataset = createDataLoaderAndDataSet(formated_full_gt_df, 
                                                         'test',
                                                         hyp_arg=hyp,
                                                         imgsz=IMG_SIZE, 
                                                         batch_size=batch_size, 
                                                         augment=False,
                                                         nw=num_workers)

restore='YoloV5Last.pt'
model_cfg_file = 'yolov5s_custom_anchors.yaml'
SHOULD_I_TRAIN = False

if SHOULD_I_TRAIN:
    model = train(20, train_loader, test_loader, device, imgsz=IMG_SIZE, restore=restore,
                  model_cfg_path=model_cfg_file)
else:
    model = Model(cfg=model_cfg_file, ch=3, nc=1)
    model.load_state_dict(torch.load(restore, map_location=device))

model.info()

In [ ]:
os.cpu_count()

In [ ]:
import random
from datetime import datetime
now = datetime.now

restore='YoloV5Last.pt'
model_cfg_file = 'yolov5s_custom_anchors.yaml'

import logging
logger = logging.getLogger()
model = Model(cfg=model_cfg_file, ch=3, nc=1);
model.load_state_dict(torch.load(restore, map_location=device))

detectInterface = makeDetectFromModel(model)

In [ ]:
from utils.datasets import LoadImages
img_, labels_out_, filepath_, shapes_ = test_dataset[155] # random.randrange(0, len(test_dataset))]

img_size = 640
dataset = LoadImages(filepath_, img_size=img_size, auto=False)
# model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # or yolov5m, yolov5l, yolov5x, custom

device = 'cuda'

imgsz = (img_size, img_size)

detectInterface.warmup(imgsz=(1, 3, imgsz))
detectInterface.to(device)

for path, im, im0s, vid_cap, s in dataset:
    
    print(im.shape)
    print(im0s)
    im0s_orig = im0s
    im = torch.from_numpy(im).float()
    im /= 255
    im = im[None, ...]
    im = im.to(device)
    t0 = now()
    
    pred = detectInterface(im)
    print('process dT =', now() - t0)
    data = detectInterface.translatePreds(pred, im.shape[2:], im0s.shape, conf_thres=0.05)
    
    
    
    im0s = cv2.cvtColor(im0s, cv2.COLOR_BGR2RGB)
    for i in range(data['count']):
        im0s = cv2.rectangle(im0s, (data['coords'][i][0], data['coords'][i][1]), 
                        (data['coords'][i][2], data['coords'][i][3]), 
                        (255, 0, 0), 
                        3)
        
        im0s = cv2.putText(im0s, str(round(data['confs'][i], 3)), 
                           (data['coords'][i][2] - 40, data['coords'][i][3]),
                           cv2.FONT_HERSHEY_SIMPLEX,
                           0.7, (255, 255, 0),
                           5, cv2.LINE_AA
                          )
        
        im0s = cv2.putText(im0s, str(round(data['confs'][i], 3)), 
                           (data['coords'][i][2] - 40, data['coords'][i][3]),
                           cv2.FONT_HERSHEY_SIMPLEX,
                           0.7, (0, 0, 0),
                           2, cv2.LINE_AA
                          )
        
        

    print('plot dT =', now() - t0)
    
fig, axs = plt.subplots(2, figsize=(20, 20))
axs[0].imshow(im0s)

im0s_orig = cv2.cvtColor(im0s_orig, cv2.COLOR_BGR2RGB)
axs[1].imshow(im0s_orig)